In [5]:
%load_ext watermark
%watermark

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
2019-04-11T03:53:16-05:00

CPython 3.7.3rc1
IPython 7.3.0

compiler   : MSC v.1916 64 bit (AMD64)
system     : Windows
release    : 10
machine    : AMD64
processor  : Intel64 Family 6 Model 142 Stepping 9, GenuineIntel
CPU cores  : 4
interpreter: 64bit


# Scraping sencillo

### 1. pandas.read_clipboard

Pandas tiene una función que permite copiar directamente una tabla de una página web a un dataframe

Por ejemplo, podemos ir a [ésta página](https://es.wikipedia.org/wiki/Anexo:Ciudades_europeas_por_poblaci%C3%B3n) que contiene una lista de las ciudades más pobladas de Europa, copiar los datos y pegarlos en un dataframe con `pd.read_clipboard`

In [6]:
import pandas as pd

In [9]:
df = pd.read_clipboard()

In [10]:
df.head(10)

,Europa,UE,País,Nombre en español,Nombre en idioma original,País .1,Entidad administrativa,Subentidad administrativa,Población
0,1,-,TUR-01,Estambul,İstanbul,Bandera de Turquía Turquía,Provincia de Estambul,NaN,14 657 434
1,2,-,RUS-0001,Moscú,Москва,Flag of Russia.svg Rusia,Distrito federal Central,Ciudad federal de Moscú,12 380 664
2,3,UE-001,GBR-01,Londres,London,Bandera de Reino Unido Reino Unido,Inglaterra,Gran Londres,8 787 892
3,4,-,RUS-0002,San Petersburgo,Санкт-Петербург,Flag of Russia.svg Rusia,Distrito federal del Noroeste,Ciudad federal de San Petersburgo,5 281 579
4,5,UE-002,ALE-01,Berlín,Berlin,Flag of Germany.svg Alemania,Berlín,NaN,3 469 849
5,6,UE-003,ESP-01,Madrid,NaN,Flag of Spain.svg España,Comunidad de Madrid,NaN,3 165 541
6,7,-,UKR-01,Kiev,Київ,Flag of Ukraine.svg Ucrania,Óblast de Kiev,NaN,2 907 684
7,8,UE-004,ITA-01,Roma,NaN,Flag of Italy.svg Italia,Lacio,Ciudad metropolitana de Roma Capital,2 864 348
8,9,UE-005,FRA-01,París,Paris,Flag of France.svg Francia,Isla de Francia,NaN,2 243 739
9,10,-,AZE-01,Bakú,Bakı,Bandera de Azerbaiyán Azerbaiyán,-,Bakú,2 181 800


In [11]:
df.to_csv("poblacion.csv", index=False)

### 2. requests + extruct

En ciertos casos, las páginas web anotan su html con información adicional para ayudar a que las máquinas puedan leer sus datos, con el objetivo generalmente de ayudar a los motores de búsqueda a indexar sus productos. Esto se llama [la web semántica](https://es.wikipedia.org/wiki/Web_sem%C3%A1ntica)). 

Por ejemplo, si buscamos un producto en google, generalmente veremos productos recomendados directamente en la página de búsqueda. Esto es así por que las paginas que tienen dichos productos tienen un conjunto de etiquetas especiales.

Podemos usar la librería [extruct](https://github.com/scrapinghub/extruct) para extraer dichos datos de forma sencilla en aquellos casos donde la página proporcione las etiquetas semánticas.

Extruct se instala de forma sencilla con `pip install extruct`

Por ejemplo, supongamos que queremos extraer información de teléfonos móviles de la tienda online [Pc Componentes](https://www.pccomponentes.com/smartphone-moviles
).

Como dicha tienda online tiene etiquetados sus productos para la web semántica, es muy facil el obtener los datos

In [51]:
import requests
import extruct

respuesta = requests.get("https://mexitel.sre.gob.mx")
datos_tienda = extruct.extract(respuesta.text)

In [52]:
from pprint import pprint
pprint(datos_tienda)

{'json-ld': [], 'microdata': [], 'microformat': [], 'opengraph': [], 'rdfa': []}


In [54]:
import requests
import extruct

r = requests.get("https://mexitel.sre.gob.mx")
datos_receta = extruct.extract(r.text)

In [55]:
from pprint import pprint
pprint(datos_receta)

{'json-ld': [], 'microdata': [], 'microformat': [], 'opengraph': [], 'rdfa': []}


In [56]:
datos_receta.keys()

dict_keys(['microdata', 'json-ld', 'opengraph', 'microformat', 'rdfa'])

In [58]:
pprint(datos_receta["microdata"][0]["properties"])

IndexError: list index out of range

In [61]:
def procesar_recetas_net(receta_url):
    r = requests.get(receta_url)
    datos_receta = extruct.extract(r.text)
    receta = datos_receta["microdata"][0]["properties"]
    return {
        "instrucciones": ''.join(receta["recipeInstructions"]),
        "ingredientes": receta["recipeIngredient"],
        "nombre": receta["name"]
    }

In [62]:
receta_nueva = procesar_recetas_net("https://mexitel.sre.gob.mx")
pprint(receta_nueva)

IndexError: list index out of range

### 3. requests + parsel

`extruct` es una herramienta muy potente cuando es posible el utilizarla. Sin embargo, el usar extruct significa que sólo podemos extraer aquellos datos que los creadores de la página han decidido que merece la pena especificar.

Por ejemplo, en la página de `recetas.net`, hay información de los platos relativa a si un plato es vegetariano o no. Dicha información no está etiquetada semánticamente y por lo tanto, extruct no la puede procesar.

Para todos aquellos casos en los que necesitemos extraer información de forma flexible, tenemos que procesar el html de forma "manual". Esto se hace especificando a nuestro script que elementos de la estructura de la página web queremos.

Una herramienta que nos permite hacer esto de forma sencilla es [`parsel`](https://github.com/scrapy/parsel). Es similar a otra herramienta más famosa `beautifulsoup`, pero en mi opinión `parsel` es más sencillo de usar.

Se instala con pip de la forma habitual (`pip install parsel` *desde fuera del notebook*).

In [30]:
import requests
from parsel import Selector

url = "http://www.recetas.net/receta/266/Spaguetti-a-la-carbonara"
r = requests.get(url)
sel = Selector(r.text)

ConnectionError: ('Connection aborted.', OSError(0, 'Error'))

In [31]:
sel

NameError: name 'sel' is not defined

Con parsel hay varias formas de seleccionar elementos específicos del html. En concreto las más potentes son mediante:

- selectores css (usando el metodo `.css`): Los selectores css son una syntaxis que nos permite hacer selecciones complejas en el html basandose en los atributos de cada elemento, en particular en sus clases de estilo css
- xpath (usando el método `.xpath`): XPath es otra forma similar a los selectores css que nos permite hacer selecciones complejas de html.

Por ejemplo si queremos seleccionar todos los elementos de tipo `div`, lo hacemos asi:

In [32]:
sel.css("div")

NameError: name 'sel' is not defined

El método `.css` siempre devuelve una lista, aunque solo haya un elemento que cumpla las condiciones de seleccion.

Podemos concatenar selecciones facilmente, por ejemplo, si queremos todos los elementos de tipo `li` que están dentro de un tipo `div` lo hacemos asi:

In [33]:
sel.css("div li")

NameError: name 'sel' is not defined

Si queremos extraer el html de una seleccion lo hacemos con el método `extract`:

In [34]:
print(sel.css("div li").extract()[0])

NameError: name 'sel' is not defined

In [35]:
type(sel.css("div li")[0])

NameError: name 'sel' is not defined

In [36]:
type(sel.css("div li").extract()[0])

NameError: name 'sel' is not defined

El método `extract` siempre devuelve una lista, aunque haya solo un elemento que cumpla el criterio de selección.
Si solo queremos extraer un elemento, podemos usar `extract_first`

In [37]:
sel.css("div li").extract_first()

NameError: name 'sel' is not defined

El selector `div li` va a seleccionar todos los `li` que estén dentro de una etiqueta `div`. Sin embargo, si queremos solo aquellos elementos **inmediatamente dentro** de una etiqueta (lo que se llama *hijos*) `div` lo hacemos asi:

In [38]:
sel.css("div>ul>li")

NameError: name 'sel' is not defined

que no devuelve nada por que no hay ningun elemento `li` inmediatamente dentro de un `div`

Si queremos seleccionar un elemento que tenga una clase específica, lo hacemos con la sintaxis, `css("elemento.clase")`

Supongamos que queremos extraer los ingredientes de la receta, si vemos el html de la página, la seccion que nos interesa tiene la forma:

```
<div class="col col-12 col-sm-8">
  <article>
    <header>
      <hgroup>
        <span id="ContentPlaceHolder1_LMetaTipoPlato" itemprop="recipeCategory" style="display:none;">Pastas y arroces</span>
        <h2 itemprop="description">
          <a id="ContentPlaceHolder1_HLTipoPlato" href="../../tipo-plato-busqueda/14/Pastas-y-arroces">Pastas y arroces</a>
          <span id="ContentPlaceHolder1_LAuthor" itemprop="author" style="display:none;">Medialabs</span>
        </h2>
        <h1 itemprop="name">
           SPAGUETTI A LA CARBONARA
        </h1>
      </hgroup>
    </header>
    <div class="ingredientes">
      <h3> INGREDIENTES PARA 4 PERSONAS</h3>
      <ul>
        <li itemprop="recipeIngredient">20 gramos de  aceite </li>
        <li itemprop="recipeIngredient">1 pizca de  sal </li>
        <li itemprop="recipeIngredient">4  huevo, las yemas </li>
        <li itemprop="recipeIngredient">1 pizca de  pimienta negra </li>
        <li itemprop="recipeIngredient">150 gramos de  queso pecorino, Grana Padano </li>
        <li itemprop="recipeIngredient">400 gramos de  spaguetti </li>
        <li itemprop="recipeIngredient">120 gramos de  tocino ahumado </li>
      </ul>
   </div>
```

O sea que hay un elemento `div` con la clase *ingredientes* que contiene dentro todos el listado de ingredientes (*los elementos `li` son elementos de una lista (list items)*)

Podemos seleccionar dicho div de la forma siguiente:

In [39]:
print(sel.css("div.ingredientes li").extract_first())

NameError: name 'sel' is not defined

Ahora dentro de este div, para conseguir los ingredientes, tenemos que tomar todos los elementos de la lista `li` y extraer su texto. Podemos extraer el texto usando el selector `::text`

In [40]:
ingredientes = sel.css("div.ingredientes li::text").extract()
ingredientes

NameError: name 'sel' is not defined

Supongamos tambien que queremos la puntuación, dificultad media, información sobre si la receta es vegetariana o no y el tiempo de preparación. Toda esta información está en una barra lateral izquierda.

El HTML tiene este aspecto:

```

            <div class="otrosDet">
              <div class="vota easing"> <span id="closeVota" class="close">X</span>
                <h2>Vota por esta receta</h2>
                <div class="score">
                  <div class="graf">
                    <div class="star" data-valor-voto="1">
                      <div data-tipo="estrella-voto" class="star"></div>
                    </div>
                    <div class="star" data-valor-voto="2">
                      <div data-tipo="estrella-voto" class="star"></div>
                    </div>
                    <div class="star" data-valor-voto="3">
                      <div data-tipo="estrella-voto" class="star"></div>
                    </div>
                    <div class="star" data-valor-voto="4">
                      <div data-tipo="estrella-voto" class="star"></div>
                    </div>
                    <div class="star" data-valor-voto="5">
                      <div data-tipo="estrella-voto" class="star"></div>
                    </div>
                  </div>
                </div>
              </div>
              <input type="hidden" name="ctl00$ContentPlaceHolder1$HFNumeroComensalesIncial" id="HFNumeroComensalesIncial" value="4" />
              <input type="hidden" name="ctl00$ContentPlaceHolder1$HFIdReceta" id="HFIdReceta" value="266" />
              <p>Dificultad: <strong>
                Media
                </strong></p>
              <p>Tiempo: <strong>
                <meta id="ContentPlaceHolder1_LMetaCookTime" itemprop="cookTime" content="PT30M"></meta>
                30
                min.</strong></p>
              <p>Vegetariana: <strong>
                No
                </strong></p>
              <p>Calorías: <strong>
                Medio
                </strong></p>
```

Vemos que la información que nos interesa está en un div de clase `otrosDet`. Dentro de dicho div, hay párrafos (etiqueta `p`) y dentro hay elementos `strong` (se usan para poner texto en negrita), con el texto que nos interesa dentro.

In [41]:
detalles = sel.css("div.otrosDet p strong::text").extract() 
detalles

NameError: name 'sel' is not defined

Vemos que estos elementos tienen un monton de *basura*, es decir, un monton de espacios y de simbolos (`\n y \r`) que no nos interesan, asi que los removemos.

In [42]:
detalles = list(map(lambda d: d.replace("\r\n                ",""), detalles))
detalles

NameError: name 'detalles' is not defined

De la misma forma podemos obtener la categoria de la receta

Las instrucciones están en el div con clase `elaboracion`, en un conjunto de parrafos `p`:

In [43]:
instrucciones = sel.css("div.elaboracion p::text").extract()
instrucciones

NameError: name 'sel' is not defined

Vemos que hay unos cuantos elementos que no necesitamos (`\r`, `\n` y espacios) al inicio y final de cada cadena de texto. Los podemos remover con `strip`

In [44]:
instrucciones = list(map(lambda x: x.strip("\r\n "),instrucciones))
instrucciones

NameError: name 'instrucciones' is not defined

Ahora podemos unir los textos con `join`

In [45]:
instrucciones = ''.join(instrucciones)
instrucciones

NameError: name 'instrucciones' is not defined

La puntuación está en el mismo `div` que los detalles de la receta, solo que en otro div de clase `score`

In [46]:
sel.css("div.otrosDet div.score span.num::text").extract_first().strip()

NameError: name 'sel' is not defined

Ahora podemos ponerlo todo en una función que nos permita extraer una receta

In [47]:
def procesar_receta(url):
    r  = requests.get(url)
    sel = Selector(r.text)
    
    categoria = sel.css("h2 a::text").extract_first() 
    titulo = sel.css("section.receta h1::text").extract_first().strip().capitalize() 
    
    instrucciones = sel.css("div.elaboracion p::text").extract()
    instrucciones = list(map(lambda x: x.strip("\r\n "),instrucciones))
    instrucciones = ''.join(instrucciones)
   
    ingredientes = sel.css("div.ingredientes li::text").extract()

    puntuacion = sel.css("div.otrosDet div.score span::text").extract_first().strip()
    
    detalles = sel.css("div.otrosDet p strong::text").extract() 
    detalles = list(map(lambda d: d.replace("\r\n                ",""), detalles))

    return {
        "categoria": categoria,
        "ingredientes": ingredientes,
        "instrucciones": instrucciones,
       "titulo": titulo,
       "puntuacion": puntuacion, 
       "dificultad": detalles[0],
       "tiempo": detalles[2],
       "vegetariana": detalles[3],
       "calorias": detalles[4]
        }

In [48]:
receta = procesar_receta("http://www.recetas.net/receta/266/Spaguetti-a-la-carbonara")

ConnectionError: ('Connection aborted.', OSError(0, 'Error'))

In [49]:
pprint(receta)

NameError: name 'receta' is not defined

In [50]:
pprint(procesar_receta("http://www.recetas.net/receta/40/Rollo-de-carne"))

ConnectionError: ('Connection aborted.', OSError(0, 'Error'))